In [3]:
from __future__ import print_function
import boto3
import os
import pandas as pd
import json
import time
from decimal import Decimal
from boto3.dynamodb.conditions import Key, Attr

In [4]:
dynamodb = boto3.resource("dynamodb", region_name = "us-west-1")

## Creates table with hash and range keys

In [47]:
table = dynamodb.create_table(
    TableName='jobs',
    KeySchema=[
        {
            'AttributeName': 'job_title',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'company_location',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'job_title',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'company_location',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 20,
        'WriteCapacityUnits': 20
    }
)


In [7]:

# Wait until the table exists.
#table.meta.client.get_waiter('table_exists').wait(TableName='jobs')

# Print out some data about the table.
print(table.item_count)

20979


In [6]:
table = dynamodb.Table("jobs")

## Add an item to the table

In [ ]:
table.put_item(
    Item = {
        "job_title" : "Data Analyst",
        "company_location" : "Indeed_Seattle",
        "company" : "Indeed",
        "location" : "Seattle",
        "city" : "WA",
        "description" : "With one application you can be considered for thousands of tech roles from leading companies on Seen. Seen by Indeed is a free service that connects you to opportunities that take you further in your career."
    })

## Get an item from the table

In [105]:
response = table.get_item(
    Key={
        "job_title": "Product Analyst",
        "company_location": "Digit_San Francisco"
    }
)
print(response['Item'])

{'city': 'San Francisco', 'job_title': 'Product Analyst', 'company': 'Digit', 'salary': None, 'zip_code': None, 'source': 'Indeed', 'reviews': None, 'description': "The Role\n--------\n\nDigit is the smartest way for anyone to prepare financially for life's needs, wants, surprises and dreams, without changing their daily behavior. This is important because managing personal finances is hard. 65% of Americans couldn't come up with $2,000 in event of an emergency. National credit card debt is at a record high of over $1 trillion. Our mission is to make financial health effortless for everyone.\n\nWe are looking for a passionate, curious and creative problem solver to be a Product Analyst at Digit. Your role will be focused on democratizing product insights and using data to drive product strategy. If you're passionate about making a meaningful difference in people's lives in partnership with a smart, driven and fun-loving team, we're excited to meet you.\n\nWhat You Offer Digit\n--------

## Query from the table using key

In [97]:
response = table.query(
    KeyConditionExpression = Key("job_title").eq("Product Analyst"))

In [98]:
for i in response['Items']:
    print(i['job_title'], ":", i['company'])

Product Analyst : 1-800-Flowers
Product Analyst : 1800flowers.com
Product Analyst : American Express7
Product Analyst : Amplify Education
Product Analyst : Arcadia.io
Product Analyst : Blend Labs
Product Analyst : Cotiviti
Product Analyst : Credible
Product Analyst : Digit
Product Analyst : Disney Parks
Product Analyst : Dom and Tom
Product Analyst : HubSpot
Product Analyst : Ipreo
Product Analyst : Kin Insurance
Product Analyst : Kiva.org
Product Analyst : Klaviyo
Product Analyst : Liftoff
Product Analyst : MCT Technology
Product Analyst : Oak Street Health
Product Analyst : Plymouth Rock Assurance
Product Analyst : Plymouth Rock Assurance
Product Analyst : Republic
Product Analyst : SmartAsset
Product Analyst : Supplyframe
Product Analyst : Swift Navigation
Product Analyst : University of San Francisco
Product Analyst : Western Asset
Product Analyst : viagogo


In [48]:
# Reads the job listing data collected from monster website
data1 = pd.read_csv("moster_data_combined.csv", usecols = [1, 2, 3, 4, 5, 6, 7, 8, 9], engine= 'python')

In [72]:
# Reads the job listing data collected from indeed website
data1 = pd.read_csv("indeed_job_listings.csv", usecols = [1, 2, 3, 4, 5, 6, 7, 8, 9], engine= 'python')

In [73]:
# Creates the range key by combining company and location columns using a "_"
data1["company_location"] = data1["Company"] + "_" + data1["City"]

In [75]:
# Renaming the columns of the dataframe
data1.columns = ["job_title", "company", "reviews", "description", "salary", "city", "state", "zip_code", "source", "company_location"]

In [76]:
len(data1)

6010

In [77]:
data1.head()

,job_title,company,reviews,description,salary,city,state,zip_code,source,company_location
0,Data Analyst,Seen by Indeed,NaN,With one application you can be considered for...,NaN,Seattle,WA,NaN,Indeed,Seen by Indeed_Seattle
1,Business Analyst,Microsoft6,686 reviews,Microsoft Cloud Hardware Infrastructure and En...,NaN,Redmond,WA,NaN,Indeed,Microsoft6_Redmond
2,ML Data Associate III,Amazon.com Services LLC46,789 reviews,Basic Qualifications\nBachelors degree or comm...,NaN,Seattle,WA,NaN,Indeed,Amazon.com Services LLC46_Seattle
3,Data Analyst,Gupta Media,NaN,Gupta Media is looking for a Data Analyst to j...,NaN,Boston,MA,2116,Indeed,Gupta Media_Boston
4,Implementation Specialist / Business Analyst,ScerIS,NaN,ScerIS is looking to add experienced and entry...,"$65,553.00 to $120,000.00 /year",Marlborough,MA,1752,Indeed,ScerIS_Marlborough


In [78]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [79]:
#Removes non ascii characters from title and description
data1["job_title"] = data1["job_title"].apply(removeNonAscii)
data1["description"] = data1["description"].apply(removeNonAscii)

In [80]:
# Converts the rows into json string
data1 = data1.to_json(orient='records')

In [81]:
# Convert json string to json object
job_data = json.loads(data1, parse_float = Decimal)

In [82]:
len(job_data)

6010

## Uploading data to the database using batch writer

In [ ]:
with table.batch_writer(overwrite_by_pkeys=['job_title', 'company_location']) as batch:
    for i in range(984, len(job_data)):
        print(i)
        batch.put_item(Item = job_data[i])
        # wait time is used between each records to reduce the write frequency in the database
        time.sleep(0.5)

In [8]:
dynamoDBClient = boto3.client('dynamodb')
table = dynamoDBClient.describe_table(
    TableName='jobs'
)
print(table)

{'Table': {'AttributeDefinitions': [{'AttributeName': 'company_location', 'AttributeType': 'S'}, {'AttributeName': 'job_title', 'AttributeType': 'S'}], 'TableName': 'jobs', 'KeySchema': [{'AttributeName': 'job_title', 'KeyType': 'HASH'}, {'AttributeName': 'company_location', 'KeyType': 'RANGE'}], 'TableStatus': 'ACTIVE', 'CreationDateTime': datetime.datetime(2020, 3, 15, 0, 5, 24, 266000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 20, 'WriteCapacityUnits': 20}, 'TableSizeBytes': 80463776, 'ItemCount': 20979, 'TableArn': 'arn:aws:dynamodb:us-west-1:764210939372:table/jobs', 'TableId': '4b853d1e-72fb-4e22-8387-247002993394'}, 'ResponseMetadata': {'RequestId': '62FU0TTS1IBVIRI4KF6EESRHUBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 15 Mar 2020 21:50:45 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '594', 'connection': 'keep-alive', 'x-amzn-requestid': '62FU0T